# AAAIM Evaluation Test - NCBI Gene

This notebook tests both single model evaluation and batch evaluation of multiple models.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Add the project root to the Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import AAAIM functions
from core import annotate_model, curate_model
from utils.evaluation import (
    evaluate_single_model,
    evaluate_models_in_folder,
    print_evaluation_results,
    compare_results
)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# LLM configuration
# llm_model = "meta-llama/llama-3.3-70b-instruct:free"  # or "gpt-4o-mini"
llm_model = "Llama-3.3-70B-Instruct"

# Evaluation parameters
max_entities_per_model = 10  # Limit entities per model for testing
num_models_to_test = 5  # Number of models to test in batch evaluation

# Entity and database configuration
entity_type = "gene"
database = "ncbigene"

output_dir = "./results/"  # Output directory for results

## Test 1: Single Model Evaluation

Evaluation of a single model with existing annotations.

In [5]:
# test_model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/119_JNK-PATHWAY_source.sbml"
test_model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/iAF692.xml"
# tax_id = 511145
tax_id = 99287
# Check if test model exist

if os.path.exists(test_model_file):
    print(f"✓ Test model found: {test_model_file}")
else:
    print(f"✗ Test model not found: {test_model_file}")

✓ Test model found: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/iAF692.xml


In [ ]:
# Test with a single model
recommendations_df, metrics = annotate_model(
    model_file=test_model_file,
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    tax_id=tax_id
)

In [9]:
recommendations_df

,file,type,id,display_name,annotation,annotation_label,match_score,existing,update_annotation
0,e_coli_core.xml,gene,G_b1241,adhE,NCBIGENE:945837,adhE,0.666667,1,keep
1,e_coli_core.xml,gene,G_b0351,mhpF,NCBIGENE:945008,mhpF,0.666667,1,keep
2,e_coli_core.xml,gene,G_s0001,s0001,,,0.000000,0,ignore
3,e_coli_core.xml,gene,G_b1849,purT,NCBIGENE:946368,purT,0.333333,1,keep
4,e_coli_core.xml,gene,G_b3115,tdcD,NCBIGENE:947635,tdcD,0.666667,1,keep
...,...,...,...,...,...,...,...,...,...
134,e_coli_core.xml,gene,G_b0008,talB,NCBIGENE:944748,talB,0.666667,1,keep
135,e_coli_core.xml,gene,G_b2464,talA,NCBIGENE:947006,talA,0.666667,1,keep
136,e_coli_core.xml,gene,G_b2465,tktB,NCBIGENE:945865,tktB,0.666667,1,keep
137,e_coli_core.xml,gene,G_b2935,tktA,NCBIGENE:947420,tktA,0.666667,1,keep


In [10]:
metrics

{'total_entities': 137,
 'entities_with_predictions': 136,
 'annotation_rate': 0.9927007299270073,
 'total_predictions': 138,
 'matches': 136,
 'accuracy': 1.0,
 'total_time': 24.440207958221436,
 'llm_time': 23.288011074066162,
 'search_time': 0.17545008659362793}

In [6]:
# Test using utils evaluation function
result_df = evaluate_single_model(
    model_file=test_model_file,
    tax_id=tax_id,
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    save_llm_results=True,
    output_dir=output_dir
)

INFO Evaluating model: iAF692.xml
INFO Using organism-specific search for tax_id: 99287
WARNING No existing annotations found in iAF692.xml


In [4]:
result_df

,model,species_id,display_name,synonyms_LLM,reason,exist_annotation_id,exist_annotation_name,predictions,predictions_names,match_score,recall_formula,precision_formula,recall_exact,precision_exact,accuracy,total_time,llm_time,query_time,tax_id,tax_name
0,e_coli_core.xml,G_b1241,ADHE,"[ADHE, adhE, EC 1.2.1.10]",The provided gene names are from the E. coli c...,[945837],adhE,[945837],[adhE],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
1,e_coli_core.xml,G_b0351,MHPF,"[MHPF, mhpF, 2-hydroxy-6-ketononatrienedioate ...",The provided gene names are from the E. coli c...,[945008],mhpF,[945008],[mhpF],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
2,e_coli_core.xml,G_b1849,PUR7,"[PUR7, purT, phosphoribosylglycinamide formylt...",The provided gene names are from the E. coli c...,[946368],purT,[946368],[purT],[0.3333333333333333],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
3,e_coli_core.xml,G_b3115,TDCD,"[TDCD, tdcD, L-tyrosine decarboxylase]",The provided gene names are from the E. coli c...,[947635],tdcD,[947635],[tdcD],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
4,e_coli_core.xml,G_b2296,ACKA,"[ACKA, ackA, acetate kinase]",The provided gene names are from the E. coli c...,[946775],ackA,[946775],[ackA],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,e_coli_core.xml,G_b0008,TALB,"[TALB, talB, transaldolase]",The provided gene names are from the E. coli c...,[944748],talB,[944748],[talB],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
132,e_coli_core.xml,G_b2464,TALA,"[TALA, talA, transaldolase]",The provided gene names are from the E. coli c...,[947006],talA,[947006],[talA],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
133,e_coli_core.xml,G_b2465,TKTB,"[TKTB, tktB, transketolase]",The provided gene names are from the E. coli c...,[945865],tktB,[945865],[tktB],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None
134,e_coli_core.xml,G_b2935,TKTA,"[TKTA, tktA, transketolase]",The provided gene names are from the E. coli c...,[947420],tktA,[947420],[tktA],[0.6666666666666666],0,0,1.0,1.0,1,22.799317,22.620837,0.17848,511145,None


## Test 2: Batch Model Evaluation

Test the evaluation of multiple models in a directory.

In [7]:
model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/'
# model_dir = "test_models"

# Check if model directory exists
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml')]
    print(f"✓ Model directory found: {model_dir}")
    print(f"  - Found {len(model_files)} XML files")
    print(f"  - Will test first {min(num_models_to_test, len(model_files))} models")

✓ Model directory found: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BiggModels/
  - Found 108 XML files
  - Will test first 5 models


In [8]:
tax_dict_file = model_dir + "bigg_organisms_no_discontinued.csv"

# Run batch evaluation 
batch_results_df = evaluate_models_in_folder(
    model_dir=model_dir,
    # num_models=min(num_models_to_test, len(model_files)),
    llm_model=llm_model,
    # max_entities=max_entities_per_model,
    entity_type=entity_type,
    database=database,
    save_llm_results=True,
    output_dir=output_dir,
    output_file="bigg_ncbigene_direct_llama.csv",
    start_at=1,
    tax_dict_file=tax_dict_file
)

Evaluating 1/108: RECON1.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/RECON1.txt
Evaluating 2/108: Recon3D.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/Recon3D.txt
Evaluating 3/108: STM_v1_0.xml
WARNING Skipping STM_v1_0.xml - no results generated
Evaluating 4/108: e_coli_core.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/e_coli_core.txt
Evaluating 5/108: iAB_RBC_283.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/iAB_RBC_283.txt
Evaluating 6/108: iAF1260.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/iAF1260.txt
Evaluating 7/108: iAF1260b.xml
LLM results saved to: results/Llama-3.3-70B-instruct-Meta/gene/iAF1260b.txt
Evaluating 8/108: iAF692.xml
WARNING Skipping iAF692.xml - no results generated
Evaluating 9/108: iAF987.xml
WARNING Skipping iAF987.xml - no results generated
Evaluating 10/108: iAM_Pb448.xml
WARNING No tax_id found for iAM_Pb448.xml
Evaluating 11/108: iAM_Pc455.xml
LL

In [12]:
if not batch_results_df.empty:
    print(f"✓ Batch evaluation successful")
    print(f"  - Evaluated {batch_results_df['model'].nunique()} models")
    print(f"  - Generated {len(batch_results_df)} total result rows")
    print(f"  - Average accuracy: {batch_results_df['accuracy'].mean():.1%}")
    
    # Show updated metrics
    print("\n  Updated metrics summary:")
    print(f"  - Average recall: {batch_results_df['recall_exact'].mean():.3f}")
    print(f"  - Average precision: {batch_results_df['precision_exact'].mean():.3f}")
    
    # Show sample of LLM results
    print("\n  Sample LLM results:")
    if 'synonyms_LLM' in batch_results_df.columns:
        for idx, row in batch_results_df[['species_id', 'synonyms_LLM']].head(3).iterrows():
            print(f"    {row['species_id']}: {row['synonyms_LLM']}")
    
    # Show match scores instead of predictions_hits
    print("\n  Sample match scores:")
    if 'match_score' in batch_results_df.columns:
        for idx, row in batch_results_df[['species_id', 'match_score']].head(3).iterrows():
            print(f"    {row['species_id']}: {row['match_score']}")
    
    # Print summary statistics
    print("\n  Summary statistics:")
    print_evaluation_results(os.path.join(output_dir, "bigg_ncbigene_direct_llama.csv"))
            

✓ Batch evaluation successful
  - Evaluated 19 models
  - Generated 20184 total result rows
  - Average accuracy: 13.5%

  Updated metrics summary:
  - Average recall: 0.135
  - Average precision: 0.108

  Sample LLM results:
    G_128_AT1: ['ADH5', 'ADH', 'ALDH']
    G_126_AT1: ['ADH1C', 'ADH', 'ALDH']
    G_9060_AT1: ['PAPSS2', 'PAPSS', 'PAPS']

  Sample match scores:
    G_128_AT1: [0.3333333333333333, 0.3333333333333333]
    G_126_AT1: [0.3333333333333333, 0.3333333333333333]
    G_9060_AT1: [0.3333333333333333, 0.3333333333333333]

  Summary statistics:
Number of models assessed: 19
Number of models with predictions: 18
Average accuracy (per model): 0.20
Ave. total time (per model): 48.64
Ave. total time (per element, per model): 0.05
Ave. LLM time (per model): 47.53
Ave. LLM time (per element, per model): 0.04
Average number of predictions per species: 0.21


## Functions test - FBC model

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import libsbml

# Add the project root to the Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import AAAIM functions
from core import annotate_model, curate_model, model_info

In [3]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml"
entity_type = "gene"
species_ids = model_info.get_all_species_ids(model_file, entity_type)
model_info.extract_model_info(model_file, species_ids, entity_type)

{'model_name': 'Raman2005 - Mycolic acid pathway of M. tuberculosis',
 'display_names': {'G_accA3': 'G_accA3',
  'G_accD3': 'G_accD3',
  'G_accD4': 'G_accD4',
  'G_accD5': 'G_accD5',
  'G_acpS': 'G_acpS',
  'G_birA': 'G_birA',
  'G_cmaA2': 'G_cmaA2',
  'G_desA1': 'G_desA1',
  'G_desA2': 'G_desA2',
  'G_desA3': 'G_desA3',
  'G_fabD': 'G_fabD',
  'G_fabG1_mabA': 'G_fabG1_mabA',
  'G_fabG2': 'G_fabG2',
  'G_fabG4': 'G_fabG4',
  'G_fabH': 'G_fabH',
  'G_fadD32': 'G_fadD32',
  'G_fas': 'G_fas',
  'G_gene42160': 'G_gene42160',
  'G_inhA': 'G_inhA',
  'G_kasB': 'G_kasB',
  'G_kasA': 'G_kasA',
  'G_mmaA1': 'G_mmaA1',
  'G_mmaA2': 'G_mmaA2',
  'G_mmaA3': 'G_mmaA3',
  'G_mmaA4': 'G_mmaA4',
  'G_pcaA': 'G_pcaA',
  'G_pks13': 'G_pks13',
  'G_UNK2': 'G_UNK2'},
 'reactions': [],
 'model_notes': 'The mycobacterial cell wall is a distinctive thick layer that protects the tubercle bacillus from general antibiotics and the host’s immune system. Mycolic acids, which are long-chain α-alkyl-β-hydroxy fatty

In [11]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioModels/BIOMD0000001046.xml"
reader = libsbml.SBMLReader()
document = reader.readSBML(model_file)
model = document.getModel()
fbc_plugin = model.getPlugin("fbc")
fbc_plugin.getPackageName()

'fbc'

In [72]:
for i in range(model.getNumReactions()):
    reaction = model.getReaction(i)
    print(reaction.getIdAttribute())
    GeneProduct = reaction.getPlugin("fbc").getGeneProductAssociation()
    if GeneProduct:
        print(GeneProduct.getAssociation().toInfix())
    else:
        print("No gene product in reaction: ", reaction.getIdAttribute())


J001
G_acpS
J002
G_birA
J003
G_accA3
J004
G_accD3
J005
G_fas
J006
G_fas
J007
G_fas
J008
G_fas
J009
G_fas
J010
G_fas
J011
G_fas
J012
G_fas
J013
G_fas
J014
G_fas
J015
G_fas
J016
G_fas
J017
G_fas
J018
G_fas
J019
G_fas
J020
G_fas
J021
G_fas
J022
G_fas
J023
G_fas
J024
G_fas
J025
G_fas
J026
G_fas
J027
G_fas
J028
G_fas
J029
G_fas
J030
G_fas
J031
G_fas
J032
G_fas
J033
G_fas
J034
G_fas
J035
G_fas
J036
G_fas
J037
G_fas
J038
G_fas
J039
G_fas
J040
G_fas
J041
G_fas
J042
G_fas
J043
G_fas
J044
G_fas
J045
G_fas
J046
G_fas
J047
G_fas
J048
G_fas
J049
G_fas
J050
G_fas
J051
G_fas
J052
G_fas
J053
G_fas
J054
G_fas
J055
G_fas
J056
G_fas
J057
G_fas
J058
G_fas
J059
G_fas
J060
G_fas
J061
G_fas
J062
G_fas
J063
G_fabD
J064
G_fabH
J065
G_fabG1_mabA
J066
G_fabG2
J067
G_fabG4
J068
G_gene42160
J069
G_inhA
J070
(G_kasB and G_kasA)
J071
G_fabG1_mabA
J072
G_fabG2
J073
G_fabG4
J074
G_gene42160
J075
G_inhA
J076
(G_kasB and G_kasA)
J077
G_fabG1_mabA
J078
G_fabG2
J079
G_fabG4
J080
G_gene42160
J081
G_inhA
J082
(G_kasB and G_

In [85]:
reaction = model.getReaction("J003")
print(reaction.getIdAttribute())
print("Reactant:")
for reactant in reaction.getListOfReactants():
    species = reactant.getSpecies()
    print(model.getSpecies(species).getName())
print("Product:")
for product in reaction.getListOfProducts():
    species = product.getSpecies()
    print(model.getSpecies(species).getName())

J003
Reactant:
ATP
BCCP-biotin
HCO3-
Product:
ADP
phosphate
BCCP-biotin-CO2
H+


## Functions test - qual model

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import libsbml

# Add the project root to the Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import AAAIM functions
from core import annotate_model, curate_model, model_info

In [2]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/119_JNK-PATHWAY_source.sbml"
entity_type = "gene"
species_ids = model_info.get_all_species_ids(model_file, entity_type)
print(species_ids)
model_info.extract_model_info(model_file, species_ids, entity_type)

['csa1', 'csa2', 'csa4', 'csa5', 'csa3', 'csa12', 'sa4', 'sa5', 'sa12', 'sa16', 'sa17', 'sa18', 'sa33', 'sa34', 'sa36', 'sa75', 'sa76', 'sa77', 'sa78']


{'model_name': 'model_id',
 'model_type': <ModelType.SBML_QUAL: 'SBML-qual'>,
 'format_info': {'has_fbc': False,
  'has_qual': True,
  'num_species': 0,
  'num_reactions': 0,
  'num_qualitative_species': 19,
  'num_transitions': 13},
 'display_names': {'csa4': 'AP-1_complex',
  'sa36': 'TP53 signalling_phenotype',
  'csa2': 'MLK1/2/3_complex',
  'sa78': 'S',
  'csa3': 'MEKK1/4_complex',
  'sa17': 'Apoptosis_phenotype',
  'csa1': 'JNK_complex_cell',
  'sa75': '3b',
  'sa34': 'TP53_phosphorylated',
  'sa12': 'BCL2_phosphorylated',
  'sa16': 'Innate Immunity_phenotype',
  'sa33': 'ATF2_phosphorylated',
  'sa18': 'Autophagy_phenotype',
  'sa5': 'MAP2K7_phosphorylated',
  'sa4': 'MAP2K4_phosphorylated',
  'csa5': 'SARS-CoV-1 proteins_complex',
  'csa12': 'JNK_complex_cell',
  'sa76': '7a',
  'sa77': '3a'},
 'reactions': ['csa1 = ((sa4 | sa5 | sa76 | sa77) & csa12) | (csa12 & sa78)',
  'csa2 = csa5',
  'csa4 = csa1 | sa75',
  'csa3 = csa5',
  'sa4 = csa3 | csa5',
  'sa5 = csa2 | csa5',
  'sa

In [3]:
prompt = model_info.format_prompt(model_file, species_ids, entity_type)
print(prompt)

Now annotate these:
Gene to annotate: csa1, csa2, csa4, csa5, csa3, csa12, sa4, sa5, sa12, sa16, sa17, sa18, sa33, sa34, sa36, sa75, sa76, sa77, sa78
Model: "model_id" 
// Display Names:
{'csa4': 'AP-1_complex', 'sa36': 'TP53 signalling_phenotype', 'csa2': 'MLK1/2/3_complex', 'sa78': 'S', 'csa3': 'MEKK1/4_complex', 'sa17': 'Apoptosis_phenotype', 'csa1': 'JNK_complex_cell', 'sa75': '3b', 'sa34': 'TP53_phosphorylated', 'sa12': 'BCL2_phosphorylated', 'sa16': 'Innate Immunity_phenotype', 'sa33': 'ATF2_phosphorylated', 'sa18': 'Autophagy_phenotype', 'sa5': 'MAP2K7_phosphorylated', 'sa4': 'MAP2K4_phosphorylated', 'csa5': 'SARS-CoV-1 proteins_complex', 'csa12': 'JNK_complex_cell', 'sa76': '7a', 'sa77': '3a'}
// Boolean Transitions:
csa1 = ((sa4 | sa5 | sa76 | sa77) & csa12) | (csa12 & sa78)
csa2 = csa5
csa4 = csa1 | sa75
csa3 = csa5
sa4 = csa3 | csa5
sa5 = csa2 | csa5
sa12 = csa1
sa16 = csa4
sa17 = !sa12
sa18 = sa12
sa33 = csa1
sa34 = csa1
sa36 = sa34
// Notes:


Return up to 3 standardized n

In [4]:
model_file = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine/218_TCA-CYCLE_source.sbml"
reader = libsbml.SBMLReader()
document = reader.readSBML(model_file)
model = document.getModel().getPlugin("qual")
model.getPackageName()

'qual'

In [ ]:
species_ids = model_info.get_all_species_ids(model_file, entity_type)
model_info.extract_model_info(model_file, species_ids, entity_type)

['csa1916', 'csa1897', 'csa1919', 'csa1898', 'csa1902', 'csa1899', 'csa1914', 'csa1949', 'csa1901', 'csa1921', 'csa1910', 'csa1895', 'csa1896', 'sa11864', 'sa12297', 'sa11889', 'sa11857', 'sa11975', 'sa11930', 'sa11896', 'sa11962', 'sa11966', 'sa11863', 'sa12216', 'sa11967', 'sa11923', 'sa11907', 'sa11979', 'sa11856', 'sa11976', 'sa11955', 'sa11883', 'sa11973', 'sa11932', 'sa11855', 'sa11946', 'sa11940', 'sa11935', 'sa11914', 'sa12001', 'sa11881', 'sa12294', 'sa11887', 'sa11945', 'sa11901', 'sa11950', 'sa11922', 'sa11980', 'sa11987', 'sa11852', 'sa11902', 'sa12024', 'sa11871', 'sa11944', 'sa11865', 'sa11867', 'sa12015', 'sa11908', 'sa11854', 'sa11964', 'sa11939', 'sa11938', 'sa12293', 'sa11929', 'sa12295', 'sa11972', 'sa11948', 'sa11927']


{'model_name': 'model_id',
 'model_type': <ModelType.SBML_QUAL: 'SBML-qual'>,
 'format_info': {'has_fbc': False,
  'has_qual': True,
  'num_species': 0,
  'num_reactions': 0,
  'num_qualitative_species': 68,
  'num_transitions': 34},
 'display_names': {'sa12024': 'pyruvate_simple_molecule_neuron',
  'csa1899': 'Succinyl-CoA_space_ligase_complex_mitochondrial_space_matrix',
  'csa1895': 'pyruvate_space_dehydrogenase_space_complex_complex_mitochondrial_space_matrix',
  'sa11939': 'oxaloacetate(2-)_simple_molecule',
  'csa1897': 'malate_space_dehydrogenase_space_2_complex',
  'sa11889': 'fumaric_space_acid_simple_molecule',
  'sa11962': 'GLUD1',
  'sa12294': 'GTP_simple_molecule',
  'sa11979': 'PC',
  'sa11901': 'ACSS1',
  'sa11987': 'ADHFE1',
  'csa1919': 'fumarate_space_hydratase_complex',
  'sa11927': 'Co2+_ion',
  'sa11863': 'GDP_simple_molecule',
  'sa11883': '(5-hydroxyindol-3-yl)acetaldehyde_simple_molecule',
  'sa11854': '(S)-malate(2-)_simple_molecule',
  'sa11935': 'succinyl-CoA

In [3]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.getenv("LLAMA_API_KEY"),base_url="https://api.llama.com/compat/v1/")
response = client.chat.completions.create(
    model="Llama-3.3-70B-Instruct",
    messages=[
        {"role": "user", "content": "Which planet do humans live on?"}
    ],
    temperature=0.2,
    max_tokens=10000)
print(response.choices[0].message.content)

Humans live on the planet Earth.
